In [ ]:
# Copyright (c) 2020 ZZH

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, MaxPool2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras import Model
import os
import numpy as np

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
epochs = 100
lr = 0.1
batch_size = 128
REGULARIZER  = 0.0001
checkpoint_save_path =  './Model/VGG/'

In [ ]:
#数据导入及数据增强
cifar10 = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
mean = [125.307, 122.95, 113.865]  #np.mean()
std = [62.9932, 62.0887, 66.7048]  #np.std()
for i in range(3):
    x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
    x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
DataGenTrain = tf.keras.preprocessing.image.ImageDataGenerator(
               rotation_range = 15,
               width_shift_range = 0.1,
               height_shift_range = 0.1,
               horizontal_flip = True,
               vertical_flip = False,
               shear_range=0.1,
               zoom_range = 0.1)
DataGenTrain.fit(x_train)

In [ ]:
#网络搭建及训练
class VGG16(Model):
    def __init__(self):
        super(VGG16,self).__init__()
        self.c1 = Conv2D(filters=64, kernel_size=(3, 3),strides=1, padding='same')
        self.b1 = BatchNormalization(momentum=0.9)
        self.a1 = Activation('relu')
        self.c2 = Conv2D(filters=64, kernel_size=(3, 3),strides=1, padding='same')
        self.b2 = BatchNormalization(momentum=0.9)
        self.a2 = Activation('relu')
        self.p1 = MaxPool2D(pool_size=(2,2),strides=2, padding='same')
        self.d1 = Dropout(0.5)

        self.c3 = Conv2D(filters=128, kernel_size=(3, 3),strides=1, padding='same')
        self.b3 = BatchNormalization(momentum=0.9)
        self.a3 = Activation('relu')
        self.c4 = Conv2D(filters=128, kernel_size=(3, 3),strides=1, padding='same')
        self.b4 = BatchNormalization(momentum=0.9)
        self.a4 = Activation('relu')
        self.p2 = MaxPool2D(pool_size=(2,2), strides=2, padding='same')
        self.d2 = Dropout(0.5)
        
        self.c5 = Conv2D(filters=256, kernel_size=(3, 3),strides=1, padding='same')
        self.b5 = BatchNormalization(momentum=0.9)
        self.a5 = Activation('relu')
        self.c6 = Conv2D(filters=256, kernel_size=(3, 3),strides=1, padding='same')
        self.b6 = BatchNormalization(momentum=0.9)
        self.a6 = Activation('relu')
        self.c7 = Conv2D(filters=256, kernel_size=(3, 3),strides=1, padding='same')
        self.b7 = BatchNormalization(momentum=0.9)
        self.a7 = Activation('relu')
        self.p3 = MaxPool2D(pool_size=(2,2), strides=2, padding='same')   
        self.d3 = Dropout(0.5)

        self.c8 = Conv2D(filters=512, kernel_size=(3, 3),strides=1, padding='same')
        self.b8 = BatchNormalization(momentum=0.9)
        self.a8 = Activation('relu')
        self.c9 = Conv2D(filters=512, kernel_size=(3, 3),strides=1, padding='same')
        self.b9 = BatchNormalization(momentum=0.9)
        self.a9 = Activation('relu')
        self.c10 = Conv2D(filters=512, kernel_size=(3, 3),strides=1, padding='same')
        self.b10 = BatchNormalization(momentum=0.9)
        self.a10 = Activation('relu')
        self.p4 = MaxPool2D(pool_size=(2,2), strides=2, padding='same')   
        self.d4 = Dropout(0.5)

        self.c11 = Conv2D(filters=512, kernel_size=(3, 3),strides=1, padding='same')
        self.b11 = BatchNormalization(momentum=0.9)
        self.a11 = Activation('relu')
        self.c12 = Conv2D(filters=512, kernel_size=(3, 3),strides=1, padding='same')
        self.b12 = BatchNormalization(momentum=0.9)
        self.a12 = Activation('relu')
        self.c13 = Conv2D(filters=512, kernel_size=(3, 3),strides=1, padding='same')
        self.b13 = BatchNormalization(momentum=0.9)
        self.a13 = Activation('relu')
        self.p5 = MaxPool2D(pool_size=(2,2), strides=2, padding='same')   
        self.d5 = Dropout(0.5)        
        
        self.flatten = Flatten()
        self.f1 = Dense(4096,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER),bias_initializer=tf.constant_initializer(0.1))
        self.d6 = Dropout(0.5)
        self.f2 = Dense(4096,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER),bias_initializer=tf.constant_initializer(0.1))
        self.d7 = Dropout(0.5)
        self.f3 = Dense(10,activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER),bias_initializer=tf.constant_initializer(0.1))
    
    def call(self,x):
        x = self.c1(x)
        x = self.b1(x)
        x = self.a1(x)
        x = self.c2(x)
        x = self.b2(x)
        x = self.a2(x)
        x = self.p1(x)
        x = self.d1(x)

        x = self.c3(x)
        x = self.b3(x)
        x = self.a3(x)
        x = self.c4(x)
        x = self.b4(x)
        x = self.a4(x)
        x = self.p2(x)
        x = self.d2(x)

        x = self.c5(x)
        x = self.b5(x)
        x = self.a5(x)
        x = self.c6(x)
        x = self.b6(x)
        x = self.a6(x)
        x = self.c7(x)
        x = self.b7(x)
        x = self.a7(x)
        x = self.p3(x)
        x = self.d3(x)

        x = self.c8(x)
        x = self.b8(x)
        x = self.a8(x)
        x = self.c9(x)
        x = self.b9(x)
        x = self.a9(x)
        x = self.c10(x)
        x = self.b10(x)
        x = self.a10(x)
        x = self.p4(x)
        x = self.d4(x)

        x = self.c11(x)
        x = self.b11(x)
        x = self.a11(x)
        x = self.c12(x)
        x = self.b12(x)
        x = self.a12(x)
        x = self.c13(x)
        x = self.b13(x)
        x = self.a13(x)
        x = self.p5(x)
        x = self.d5(x)

        x = self.flatten(x)
        x = self.f1(x)
        x = self.d6(x)
        x = self.f2(x)
        x = self.d7(x)
        y = self.f3(x)
        return y

In [ ]:
model = VGG16()

model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

log_dir = os.path.join("Model","VGG16_logs")
callbacks = [
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_lr=0.0001, patience=10, cooldown=0),
            tf.keras.callbacks.ModelCheckpoint(     #模型保存
                filepath = checkpoint_save_path,
                save_weights_only = False,
                monitor = 'val_accuracy',
                save_best_only = True),
#             tf.keras.callbacks.EarlyStopping(       #早停
#                 monitor = 'val_accuracy',
#                 patience=10, 
#                 baseline=None),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=False)  #保存计算图
]

hist = model.fit(DataGenTrain.flow(x_train,y_train,batch_size=batch_size,shuffle=True),
                 epochs=epochs,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=callbacks)

model.summary()

In [ ]:
#结果可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use({'figure.figsize':(6,4)})

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
print('best result: {:.2f}%  ({}epochs)'.format(100*max(hist.history['val_accuracy']),1+hist.history['val_accuracy'].index(max(hist.history['val_accuracy']))))

In [ ]:
#tensorboard可视化
#!tensorboard --logdir=./Model/VGG16_logs
#http://localhost:6006/